### Python Code to solve p-medians problem
### Author: Gilsiley Henrique Darú

In [ ]:
#libraries
#pandas to dataframes
#numpy to arrays
#matplotlib and seaborn to graphics
#pulp to linear programmin
#re to regular expressions (parse solver variables)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sbn
import pulp as plp
import re
import time
#import pixiedust

In [ ]:
#%%pixie_debugger
#Class with problems data
class pMedianProblem():
    def __init__(self):
        #Create a dataframe to save problem data
        self.data = pd.DataFrame(columns=['x','y','weight'])
    
    def generate_random_problem(self, size = 100, weighted = False, seed = 10):
        #Create a random problem with size points and two coordinates (x,y)
        #the random values ~N(0,1)
        np.random.seed(seed) #necessary to replicate simulation
        x = np.random.randn(size)
        y = np.random.randn(size)
        self.data['x'] = x
        self.data['y'] = y
        #generate random weigths ~N(0,1)
        self.data['weight'] = np.random.randn(size) if weighted else 1 
        
#Class to store a solution
class pMedianSolution():
    def __init__(self, problem, p=3):
        self.data = pd.DataFrame(columns=['x','y','weight','cluster','isMedian','distance'])
        self.data = self.data.append(problem.data)
        self.data['cluster'] = -1
        self.data['distance'] = -1
        self.data['isMedian']=False
        self.p = p
    
    def medians(self):
        return self.data[self.data.isMedian==1]
    
    def objective(self):
        return self.data.distance.sum()

#Class to solve a p-median problem  
class pMedianSolver():
    def __init__(self, problem, weighted = False):
        self.problem = problem
        self.weighted = weighted

                
    def plot(self, solution, title = ''):
        df = solution.data
        objective = 'Objective {:.2f}'.format(solution.objective())
        p = sbn.scatterplot(x = df.x, y=df.y, hue = df.cluster, size = (df.isMedian+1), palette = 'Dark2')
        #fonte: https://stackoverflow.com/questions/53437462/how-do-i-remove-an-attribute-from-the-legend-of-a-scatter-plot
        # EXTRACT CURRENT HANDLES AND LABELS
        h,l = p.get_legend_handles_labels()
        # COLOR LEGEND (FIRST 30 ITEMS)
        col_lgd = plt.legend(h[:4], l[:4], loc='upper left')
        # ADD FORMER (OVERWRITTEN BY LATTER)
        p.add_artist(col_lgd)
        p.set_title(title + ":" + objective)
        plt.show()
        
    def Heuristic01(self, p = 3):
        self.actual_solution = pMedianSolution(self.problem, p)
        #The first p elements is set to median
        #columns={0:'x',1:'y',2:'weight',3:'cluster',4:'isMedian',5:'distance']
        #Set isMedian equal True for 3 first points 
        self.actual_solution.data.iloc[:p,-2] = True
        #Set cluster number for 3 first points 
        self.actual_solution.data.iloc[:p,-3] = np.arange(p)
        #Set medians positions
        self.index_medians = list(range(p))
        for i in range(p):
            #Set the median coordinates
            x_median = self.actual_solution.data.iloc[self.index_medians[i],0]    
            y_median = self.actual_solution.data.iloc[self.index_medians[i],1]
            #Calculate the distance between point and current median
            self.actual_solution.data['distance_temp'] = ((self.actual_solution.data.x - x_median)**2 + (self.actual_solution.data.y-y_median)**2)
            #Set cluster if cluster equal to -1 or actual distance less than distance from old median
            self.actual_solution.data['cluster'] = self.actual_solution.data.apply(
                lambda row: i if (row.distance==-1)|(row.distance_temp<row.distance) else row.cluster, axis=1)
            #Set new distance from point to median if distance less than old distance
            self.actual_solution.data['distance'] = self.actual_solution.data.apply(
                lambda row: row.distance_temp if (row.distance==-1)|(row.distance_temp<row.distance) else row.distance, axis=1)   
        self.actual_solution.data['distance'] = np.sqrt(self.actual_solution.data['distance'])
    
    def Heuristic02(self, p = 3, report = False):
        self.actual_solution = pMedianSolution(self.problem, p)
        tol = 0.001
        max_iter = 10
        best_value = self.actual_solution.data.x.abs().max()**2+self.actual_solution.data.y.abs().max()**2
        best_value *= self.actual_solution.data.shape[0]
        improvement = tol
        iter = 0
        #initialize p-Medians with p first elements
        self.index_medians = list(range(p))
        if report:
            print('index:',self.index_medians)
            display(self.actual_solution.data.loc[self.index_medians,:])            
        while improvement >= tol and iter<max_iter:
            #Set all points without cluster, distance and isMedian
            self.actual_solution.data['cluster'] = -1
            self.actual_solution.data['distance'] = -1
            self.actual_solution.data['isMedian'] = False
            #Set medians
            self.actual_solution.data.loc[self.index_medians,'isMedian'] = True            
            #Set cluster number for p-medians
            self.actual_solution.data.loc[self.index_medians,'cluster'] = np.arange(p)
            for i in range(p):
                #Set the median coordinates
                x_median = self.actual_solution.data.iloc[self.index_medians[i],0]    
                y_median = self.actual_solution.data.iloc[self.index_medians[i],1]
                #Calculate the distance between point and current median
                self.actual_solution.data['distance_temp'] = ((self.actual_solution.data.x - x_median)**2 + (self.actual_solution.data.y-y_median)**2)
                #Set cluster if cluster equal to -1 or actual distance less than distance from old median
                self.actual_solution.data['cluster'] = self.actual_solution.data.apply(
                    lambda row: i if (row.distance==-1)|(row.distance_temp<row.distance) else row.cluster, axis=1)
                #Set new distance from point to median if distance less than old distance
                self.actual_solution.data['distance'] = self.actual_solution.data.apply(
                    lambda row: row.distance_temp if (row.distance==-1)|(row.distance_temp<row.distance) else row.distance, axis=1)   
            self.actual_solution.data['distance'] = np.sqrt(self.actual_solution.data['distance'])
            improvement = abs(best_value - self.actual_solution.objective())
            if self.actual_solution.objective() < best_value:
                old_value = best_value
                best_value = self.actual_solution.objective()
            iter += 1
            if report:
                print("Old objective value:",old_value)
                print("New objective value:",best_value)
                print("Improvement:",improvement)
                print("Iteration:",iter)                
            #Update medians nearest point from centroid
            a = self.actual_solution.data.copy()
            #Calculate means value by cluster
            groups = solver.actual_solution.data[['cluster','x','y']].groupby(by='cluster').mean()
            #For each point calculate distance up to mean
            a['distance_med'] = a.apply(lambda row: np.sqrt((row.x-groups.loc[row.cluster,:].x)**2 
                                                        + (row.y-groups.loc[row.cluster,:].y)**2), axis=1)
            a['rank'] = a[['cluster','distance_med']].groupby(by='cluster').rank(method='first')
            #find nearest point
            #min_values = a.groupby(by='cluster').agg({'distance_med':min}).reset_index()
            if report:
                display(min_values)
            #join min_values with a
            #g = pd.merge(left=a, right=min_values, left_on='cluster',right_on='cluster',how='left')
            if report:
                a.to_excel('g.xlsx')
            if report:
                print("Old medians points:",self.index_medians)
            #Select nearest points
            self.index_medians = a[a['rank']==1].index
            if report:
                print("New medians points:",self.index_medians)
                display(self.actual_solution.data.loc[self.index_medians,:])
                
                
    
    def OptimalSolution(self, p = 3):
        #parameters
        #p is the number of medians
        self.actual_solution = pMedianSolution(self.problem, p)
        #Create a linear programming variable
        opt_model = plp.LpProblem(name="p-Medianas", sense = plp.LpMinimize)
        dict_var = {'Name':[],'Object':[]}
        #Y is medians point
        #Generate Y variables
        Y = []
        for i in range(self.problem.data.shape[0]):
            name = "Y_"+str(i)
            Y.append(plp.LpVariable(name,0,1,plp.LpInteger))
        #Generate alocation variables X and distance matrix D 
        pontos = self.problem.data.shape[0]
        D = np.zeros([pontos,pontos])
        X = []
        for i in range(self.problem.data.shape[0]):
            lin = []
            for j in range(self.problem.data.shape[0]):
                name = "X[{}][{}]".format(i,j)
                lin.append(plp.LpVariable(name,0,1,plp.LpInteger))
                D[i,j] = np.sqrt((self.problem.data.iloc[i,0]-self.problem.data.iloc[j,0])**2 
                                 + (self.problem.data.iloc[i,1]-self.problem.data.iloc[j,1])**2)
            X.append(lin)
        #pd.DataFrame(D).to_excel('Matriz.xlsx')
        #Generate objective function
        opt_model += plp.lpSum( [D[i,j]*X[i][j] for i in range(pontos) for j in range(pontos)])
        #Restriction number of medians
        opt_model += plp.lpSum([Y[i] for i in range(pontos)]) == p, "Num Medianas"
        #Allocation restriction each data point in associate only one median
        for i in range(pontos):
            opt_model += plp.lpSum([X[i][j] for j in range(pontos)]) == 1, "Aloc Ponto " + str(i)
        #Allocation is permitted only medians points
        for i in range(pontos):
            for j in range(pontos):
                opt_model += X[i][j] <= Y[j] , "X[{}][{}]".format(i,j)
        key = np.random.randint(100000)
        opt_model.writeLP("pMedianas"+str(key)+".lp")
        opt_model.solve()
        self.actual_solution.data['isMedian']=False
        self.actual_solution.data['cluster']=-1
        for v in opt_model.variables():
            if v.varValue >= 0.9:
                #print(v.name, "=", v.varValue)
                coord = re.findall(r'(\d+)',v.name)
                if v.name[0]=='Y': #isMedian
                    self.actual_solution.data.loc[int(coord[0]),'isMedian']=True
                else:
                    self.actual_solution.data.loc[int(coord[0]),'cluster'] = int(coord[1])
                    self.actual_solution.data.loc[int(coord[0]),'distance'] = D[int(coord[0]),int(coord[1])] 

###  How to use
* Instantiate a variable pMedianProblem 
* Load(not implemented) or generate a random problem using variable.generate_random_problem(size)
* Create a variable pMedianSolver and pass problem
* Run a solver Heuristic or Optimal, inform number of medians
* Plot results

In [ ]:
problem = pMedianProblem()
problem.generate_random_problem()
solver = pMedianSolver(problem)
solver.Heuristic01(p=3)
sol1 = solver.actual_solution
solver.Heuristic02(p=3)
sol2 = solver.actual_solution
solver.OptimalSolution()
sol3 = solver.actual_solution
solver.plot(sol1, "Heuristic01")
solver.plot(sol2, "Heuristic02")
solver.plot(sol3, "Optimal")

### Methodology
* Was used two parameters, size of problem and number of medians.
* The size of problem was generate in range(50,500) by 50
* The number of medians was generated in set {2,3,4,5,10}

In [11]:
df[df['size']==size]

dict_keys(['size', 'p', 'sample', 'type', 'objective', 'time', 'optimal', 'time_optimal'])

In [ ]:
try: 
    df = pd.read_excel('results2.xlsx')
    dict_results = df[df.columns[1:]].to_dict('list')
except:
    dict_results = {'size':[], 'p':[],'sample':[],'type':[],'objective':[],'time':[],'optimal':[],'time_optimal':[]}
for size in range(350,501,50):
    for p in [2,3,4,5,10]:
        for sample in range(30):
            if df[(df['size']==size)&(df['p']==p)&(df['sample']==sample+1)].shape[0]==0:
                print('Size:',size,' p:',p,' sample:',sample+1)
                dict_results['size'].append(size)
                dict_results['p'].append(p)
                dict_results['sample'].append(sample+1)
                problem = pMedianProblem()
                problem.generate_random_problem(size=size, seed = sample)
                solver = pMedianSolver(problem)           
                start = time.time()
                solver.Heuristic02(p=p)
                end = time.time()
                dict_results['type'].append('Heuristic')
                dict_results['objective'].append(solver.actual_solution.objective())
                dict_results['time'].append(end - start)
                start = time.time()
                solver.OptimalSolution(p=p)
                end = time.time()
                dict_results['optimal'].append(solver.actual_solution.objective())
                dict_results['time_optimal'].append(end - start)
                results = pd.DataFrame(dict_results)
                results.to_excel('results2.xlsx')
            else:
                print('Size:',size,' p:',p,' sample:',sample+1,' já calculado.')
                

Size: 350  p: 2  sample: 1  já calculado.
Size: 350  p: 2  sample: 2  já calculado.
Size: 350  p: 2  sample: 3  já calculado.
Size: 350  p: 2  sample: 4  já calculado.
Size: 350  p: 2  sample: 5  já calculado.
Size: 350  p: 2  sample: 6  já calculado.
Size: 350  p: 2  sample: 7  já calculado.
Size: 350  p: 2  sample: 8  já calculado.
Size: 350  p: 2  sample: 9  já calculado.
Size: 350  p: 2  sample: 10  já calculado.
Size: 350  p: 2  sample: 11  já calculado.
Size: 350  p: 2  sample: 12  já calculado.
Size: 350  p: 2  sample: 13  já calculado.
Size: 350  p: 2  sample: 14  já calculado.
Size: 350  p: 2  sample: 15  já calculado.
Size: 350  p: 2  sample: 16  já calculado.
Size: 350  p: 2  sample: 17  já calculado.
Size: 350  p: 2  sample: 18  já calculado.
Size: 350  p: 2  sample: 19  já calculado.
Size: 350  p: 2  sample: 20  já calculado.
Size: 350  p: 2  sample: 21  já calculado.
Size: 350  p: 2  sample: 22  já calculado.
Size: 350  p: 2  sample: 23  já calculado.
Size: 350  p: 2  sam

Size: 400  p: 4  sample: 10
Size: 400  p: 4  sample: 11
Size: 400  p: 4  sample: 12
Size: 400  p: 4  sample: 13
Size: 400  p: 4  sample: 14
Size: 400  p: 4  sample: 15
Size: 400  p: 4  sample: 16
Size: 400  p: 4  sample: 17
Size: 400  p: 4  sample: 18
Size: 400  p: 4  sample: 19
Size: 400  p: 4  sample: 20
Size: 400  p: 4  sample: 21
Size: 400  p: 4  sample: 22
Size: 400  p: 4  sample: 23
Size: 400  p: 4  sample: 24
Size: 400  p: 4  sample: 25
Size: 400  p: 4  sample: 26
Size: 400  p: 4  sample: 27
Size: 400  p: 4  sample: 28
Size: 400  p: 4  sample: 29
Size: 400  p: 4  sample: 30
Size: 400  p: 5  sample: 1
Size: 400  p: 5  sample: 2
Size: 400  p: 5  sample: 3
Size: 400  p: 5  sample: 4
Size: 400  p: 5  sample: 5
Size: 400  p: 5  sample: 6
Size: 400  p: 5  sample: 7
Size: 400  p: 5  sample: 8
Size: 400  p: 5  sample: 9
Size: 400  p: 5  sample: 10
